# Логистическая регрессия. Log Loss

In [1]:
import numpy as np

In [2]:
# Стандартизация
def standartization(X):
    return (X - X.mean(axis=0)) / X.std(axis=0)

In [3]:
# Признаки
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 1, 3, 0, 5, 10, 1, 2],
              [500, 700, 750, 600, 1450, 800, 1500, 2000, 450, 1000],
              [21, 25, 27, 20, 25,  18, 35, 60, 20, 30]], dtype = np.float64).T
# Целевая переменная
y = np.array([0, 0, 0, 1, 1, 1, 1, 0, 0, 0], dtype = np.float64)

In [4]:
print(f'Исходные значения признаков: \n{X}')

Исходные значения признаков: 
[[1.00e+00 1.00e+00 5.00e+02 2.10e+01]
 [1.00e+00 1.00e+00 7.00e+02 2.50e+01]
 [1.00e+00 2.00e+00 7.50e+02 2.70e+01]
 [1.00e+00 1.00e+00 6.00e+02 2.00e+01]
 [1.00e+00 3.00e+00 1.45e+03 2.50e+01]
 [1.00e+00 0.00e+00 8.00e+02 1.80e+01]
 [1.00e+00 5.00e+00 1.50e+03 3.50e+01]
 [1.00e+00 1.00e+01 2.00e+03 6.00e+01]
 [1.00e+00 1.00e+00 4.50e+02 2.00e+01]
 [1.00e+00 2.00e+00 1.00e+03 3.00e+01]]


In [5]:
# Стандартизация признаков
X_st = X.copy()
X_st[:, 1:] = standartization(X[:, 1:])
X_st

print(f'Стандартизованные значения признаков: \n{X_st}')

Стандартизованные значения признаков: 
[[ 1.         -0.57142857 -0.97958969 -0.60595294]
 [ 1.         -0.57142857 -0.56713087 -0.264571  ]
 [ 1.         -0.21428571 -0.46401617 -0.09388003]
 [ 1.         -0.57142857 -0.77336028 -0.69129842]
 [ 1.          0.14285714  0.97958969 -0.264571  ]
 [ 1.         -0.92857143 -0.36090146 -0.86198939]
 [ 1.          0.85714286  1.08270439  0.58888384]
 [ 1.          2.64285714  2.11385144  2.72252095]
 [ 1.         -0.57142857 -1.08270439 -0.69129842]
 [ 1.         -0.21428571  0.05155735  0.16215642]]


### Задание 1

*Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log (как вариант - использовать np.clip или np.where).*

In [6]:
# Расчет без фильтрации
def calc_logloss_init(y, y_pred):
    err = np.mean(-y * np.log(y_pred) - (1.0 - y) * np.log(1.0 - y_pred))
    return err

Для того, чтобы значения 0 и 1 не вызывали ошибку при расчете logloss будем немного корректировать их - исправлять на подходящие для расчета значения.

In [7]:
# Расчет с коррекцией
def calc_logloss(y, y_pred):
    zero_appr = 1e-6
    one_appr = 1 - zero_appr
    
    np.clip(y_pred, zero_appr, one_appr, out=y_pred)
    
    err = np.mean(-y * np.log(y_pred) - (1.0 - y) * np.log(1.0 - y_pred))
    return err

Проверим, что результаты совпадает для "корректных" данных.

In [8]:
y_test = np.array([1, 0, 1, 0])
y_pred_test = np.array([0.9, 0.1, 0.99, 0.01])

error = calc_logloss_init(y_test, y_pred_test)
print(f'log_loss_init = {error}')

error = calc_logloss(y_test, y_pred_test)
print(f'log_loss = {error}')

log_loss_init = 0.05770542575566387
log_loss = 0.05770542575566387


Проверим расчет для "некорректных" данных.

In [9]:
y_test = np.array([1, 0, 1, 0])
y_pred_test = np.array([0.9, 0.1, 1, 0])

# Исходный вариант возвращает inf и выдает ошибку
error = calc_logloss_init(y_test, y_pred_test)
print(f'log_loss_init = {error}')

# Скорректированный вариант считает без ошибок
error = calc_logloss(y_test, y_pred_test)
print(f'log_loss = {error}')

log_loss_init = nan
log_loss = 0.05268075782916315


C:\Users\JK\AppData\Local\Temp/ipykernel_22400/2903139550.py:3: RuntimeWarning: divide by zero encountered in log
  err = np.mean(-y * np.log(y_pred) - (1.0 - y) * np.log(1.0 - y_pred))
C:\Users\JK\AppData\Local\Temp/ipykernel_22400/2903139550.py:3: RuntimeWarning: invalid value encountered in multiply
  err = np.mean(-y * np.log(y_pred) - (1.0 - y) * np.log(1.0 - y_pred))


### Задание 2

*На данных из урока изучите влияние гиперпараметров на ошибку алгоритма.*

In [10]:
# Вычисляет вероятность
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [11]:
# Логистическая регрессия
def eval_LR_model(X, y, iterations, eta=1e-4):
    np.random.seed(42)
    w = np.random.randn(X.shape[1])
    n = X.shape[0]
    for i in range(1, iterations + 1):
        z = np.dot(X, w)
        pred = sigmoid(z)
        w -= eta * (1 / n * np.dot((pred - y), X))
        err = calc_logloss(y, sigmoid(np.dot(X, w)))
        if i % (iterations / 10) == 0:
            print(f'Iteration = {i}, w = {w}, error = {err}')
    return w

In [12]:
# Вычисление весов
w = eval_LR_model(X_st, y, iterations=100, eta=0.1)

Iteration = 10, w = [ 0.36065583 -0.3972585   0.51937535  1.19161524], error = 0.972199021173242
Iteration = 20, w = [ 0.215788   -0.61251766  0.44159174  0.90102578], error = 0.8152129340815597
Iteration = 30, w = [ 0.08277136 -0.76559453  0.42551123  0.67107078], error = 0.7212678942568461
Iteration = 40, w = [-0.02507329 -0.85541749  0.46585326  0.50463174], error = 0.6722796074198796
Iteration = 50, w = [-0.10912684 -0.90576455  0.53830407  0.37884616], error = 0.6416699927595096
Iteration = 60, w = [-0.17469927 -0.93640342  0.62421319  0.27424279], error = 0.6181685428656378
Iteration = 70, w = [-0.22617113 -0.95733883  0.71436092  0.18090474], error = 0.5982903907109156
Iteration = 80, w = [-0.26681997 -0.97319728  0.80460517  0.09421158], error = 0.5807656397146188
Iteration = 90, w = [-0.29911987 -0.98614533  0.89312045  0.01196478], error = 0.5649873364632754
Iteration = 100, w = [-0.32495521 -0.99723356  0.97913034 -0.06693668], error = 0.5506011060815837


Исследуем влияние скорости обучения eta на ошибку. Зафиксируем количество итераций iterations = 100. Получим значения ошибки:

eta = 0.01, error = 0.97

eta = 0.1, error = 0.55

eta = 0.5, error = 0.33

При увеличении скорости обучения ошибка уменьшается при одном и том же количестве итерации, т.е алгоритм сходится быстрее.

Исследуем влияние количества итераций на ошибку. Зафиксируем скорость обучения eta = 0.1.

iterations = 100, error = 0.55

iterations = 500, error = 0.33

iterations = 1000, error = 0.26

При увеличении количества итераций ошибка также уменьшается и алгоритм сходится быстрее.

### Задание 3

*Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса "1". На вход функции подаются значения признаков Х и веса, которые уже посчитаны функцией eval_LR_model.*

In [13]:
# Вычисляет вероятность класса 1
def calc_pred_proba(X, w):
    return sigmoid(np.dot(X, w))

In [14]:
# Проверка
prob = calc_pred_proba(X_st, w)
print(f'prob = {prob.round(2)}')

prob = [0.34 0.43 0.36 0.39 0.62 0.58 0.46 0.25 0.32 0.48]


### Задание 4

*Создайте функцию calc_pred, возвращающую предсказанные классы (0 или 1). На вход функции подаются значения признаков Х и веса, которые уже посчитаны функцией eval_LR_model, а также порог вероятности.*

In [15]:
# Вычисляет класс (0 или 1) по предсказанной вероятности и порогу 
def calc_pred(X, w, prob_threshold=0.5):
    pred = calc_pred_proba(X, w)
    return np.where(pred > prob_threshold, 1, 0)

In [16]:
# Проверка
classes = calc_pred(X_st, w)
print(f'classes = {classes}')

classes = [0 0 0 0 1 1 0 0 0 0]
